In [0]:
peopleDF = spark.read.parquet("/FileStore/tables/people-10m.parquet")
peopleDF.show(5)

+---+---------+----------+----------+------+-------------------+-----------+------+
 id|firstName|middleName| lastName|gender| birthDate| ssn|salary|
+---+---------+----------+----------+------+-------------------+-----------+------+
 1| Pennie| Carry|Hirschmann| F|1955-07-02 04:00:00|981-43-9345| 56172|
 2| An| Amira| Cowper| F|1992-02-08 05:00:00|978-97-8086| 40203|
 3| Quyen| Marlen| Dome| F|1970-10-11 04:00:00|957-57-8246| 53417|
 4| Coralie| Antonina| Marshal| F|1990-04-11 04:00:00|963-39-4885| 94727|
 5| Terrie| Wava| Bonar| F|1980-01-16 05:00:00|964-49-8051| 79908|
+---+---------+----------+----------+------+-------------------+-----------+------+
only showing top 5 rows

In [0]:
from pyspark.sql.functions import abs, col, round
peopleWithFixedSalariesDF = peopleDF.select("firstName", "middleName", "lastName", "gender", "ssn", abs(col("salary")).alias("salary"))
peopleWithFixedSalariesDF.count()

Out[8]: 10000000

In [0]:
peopleWithFixedSalariesDF.show(3)

+---------+----------+----------+------+-----------+------+
firstName|middleName| lastName|gender| ssn|salary|
+---------+----------+----------+------+-----------+------+
 Pennie| Carry|Hirschmann| F|981-43-9345| 56172|
 An| Amira| Cowper| F|978-97-8086| 40203|
 Quyen| Marlen| Dome| F|957-57-8246| 53417|
+---------+----------+----------+------+-----------+------+
only showing top 3 rows

In [0]:
peopleWithFixedSalaries20DF = peopleWithFixedSalariesDF.select("*")\
                              .withColumn("salary10K", round(col("salary")/10000))\
                              .filter("salary >= 20000")
peopleWithFixedSalaries20DF.show(5)

+---------+----------+----------+------+-----------+------+---------+
firstName|middleName| lastName|gender| ssn|salary|salary10K|
+---------+----------+----------+------+-----------+------+---------+
 Pennie| Carry|Hirschmann| F|981-43-9345| 56172| 6.0|
 An| Amira| Cowper| F|978-97-8086| 40203| 4.0|
 Quyen| Marlen| Dome| F|957-57-8246| 53417| 5.0|
 Coralie| Antonina| Marshal| F|963-39-4885| 94727| 9.0|
 Terrie| Wava| Bonar| F|964-49-8051| 79908| 8.0|
+---------+----------+----------+------+-----------+------+---------+
only showing top 5 rows

In [0]:
from pyspark.sql import Row
l = [('Ankit', 25), ('Jalfaizy', 22), ('saurabh', 20), ('Bala', 26)]
rdd = sc.parallelize(l)
people = rdd.map(lambda x : Row(name=x[0], age=int(x[1])))
schemaPeople = sqlContext.createDataFrame(people)

In [0]:
people.take(3)

Out[13]: [Row(name='Ankit', age=25),
 Row(name='Jalfaizy', age=22),
 Row(name='saurabh', age=20)]

In [0]:
schemaPeople.show(3)

+--------+---+
 name|age|
+--------+---+
 Ankit| 25|
Jalfaizy| 22|
 saurabh| 20|
+--------+---+
only showing top 3 rows

In [0]:
geoip_rdd = sc.textFile("/FileStore/tables/text/geoip.txt")
geoip_rdd.take(3)

Out[15]: ['194.20.126.123,NL,Netherlands',
 '94.126.119.173,FR,France',
 '193.46.74.166,RU,Russian Federation']

In [0]:
geoip_rdd1 = geoip_rdd\
            .map(lambda x : x.split(","))
geoip_rdd1.take(3)

Out[16]: [['194.20.126.123', 'NL', 'Netherlands'],
 ['94.126.119.173', 'FR', 'France'],
 ['193.46.74.166', 'RU', 'Russian Federation']]

In [0]:
geoip_rdd2 = geoip_rdd1.map(lambda x : Row(ip=x[0], code=x[1], country=x[2]))
geoip_rdd2.take(3)

Out[17]: [Row(ip='194.20.126.123', code='NL', country='Netherlands'),
 Row(ip='94.126.119.173', code='FR', country='France'),
 Row(ip='193.46.74.166', code='RU', country='Russian Federation')]

In [0]:
geoipDF = sqlContext.createDataFrame(geoip_rdd2)
geoipDF.show()

+--------------+----+------------------+
 ip|code| country|
+--------------+----+------------------+
194.20.126.123| NL| Netherlands|
94.126.119.173| FR| France|
 193.46.74.166| RU|Russian Federation|
 46.235.67.202| RU|Russian Federation|
193.161.193.64| RU|Russian Federation|
+--------------+----+------------------+

In [0]:
#RDD API
geoipDF.rdd.collect()

Out[19]: [Row(ip='194.20.126.123', code='NL', country='Netherlands'),
 Row(ip='94.126.119.173', code='FR', country='France'),
 Row(ip='193.46.74.166', code='RU', country='Russian Federation'),
 Row(ip='46.235.67.202', code='RU', country='Russian Federation'),
 Row(ip='193.161.193.64', code='RU', country='Russian Federation')]

In [0]:
#RDD API
geoipDF.rdd\
  .filter(lambda x : x.country == 'Russian Federation')\
  .take(3)

Out[20]: [Row(ip='193.46.74.166', code='RU', country='Russian Federation'),
 Row(ip='46.235.67.202', code='RU', country='Russian Federation'),
 Row(ip='193.161.193.64', code='RU', country='Russian Federation')]

In [0]:
#DataFrame API
geoipDF.select("ip", "country")\
  .where("country == 'Russian Federation'")\
  .show(4)

+--------------+------------------+
 ip| country|
+--------------+------------------+
 193.46.74.166|Russian Federation|
 46.235.67.202|Russian Federation|
193.161.193.64|Russian Federation|
+--------------+------------------+

In [0]:
# SQL
geoipDF.createOrReplaceTempView("geoip")
spark.sql("select ip, country from geoip where country = 'Russian Federation'").show()

+--------------+------------------+
 ip| country|
+--------------+------------------+
 193.46.74.166|Russian Federation|
 46.235.67.202|Russian Federation|
193.161.193.64|Russian Federation|
+--------------+------------------+

In [0]:
peopleDF.createOrReplaceTempView("People10M")

In [0]:
%sql
select firstName, count(lastName) from people10M group by firstName limit 3

firstName,count(lastName)
Alayna,1313
Melaine,1237
Faye,1350


In [0]:
import pyspark.sql.functions as f
peopleDF.groupBy("firstName").agg(f.count("lastName")).show(3)

+---------+---------------+
firstName|count(lastName)|
+---------+---------------+
 Alayna| 1313|
 Melaine| 1237|
 Faye| 1350|
+---------+---------------+
only showing top 3 rows

In [0]:
peopleDF.groupBy("firstName").agg(f.count('lastName').alias('lastName count')).show(3)

+---------+--------------+
firstName|lastName count|
+---------+--------------+
 Alayna| 1313|
 Melaine| 1237|
 Faye| 1350|
+---------+--------------+
only showing top 3 rows

In [0]:
peopleDF.groupBy('firstName').agg({'lastName':'count'}).show(3)

+---------+---------------+
firstName|count(lastName)|
+---------+---------------+
 Alayna| 1313|
 Melaine| 1237|
 Faye| 1350|
+---------+---------------+
only showing top 3 rows

In [0]:
peopleDF.groupBy('firstName').agg({'lastName':'count'}).withColumnRenamed('count(lastName)', 'lastName count').show(3)

+---------+--------------+
firstName|lastName count|
+---------+--------------+
 Alayna| 1313|
 Melaine| 1237|
 Faye| 1350|
+---------+--------------+
only showing top 3 rows

In [0]:
peopleDF.groupBy(f.length('firstName')).agg(f.count('*')).show(3)

+-----------------+--------+
length(firstName)|count(1)|
+-----------------+--------+
 6| 2633717|
 3| 428721|
 5| 2278227|
+-----------------+--------+
only showing top 3 rows

In [0]:
peopleDF.groupBy(f.length('firstName')).count().show(3)

+-----------------+-------+
length(firstName)| count|
+-----------------+-------+
 6|2633717|
 3| 428721|
 5|2278227|
+-----------------+-------+
only showing top 3 rows

In [0]:
peopleDF.groupBy('firstName', f.length('firstName')).agg(f.count("*")).show(3)

+---------+-----------------+--------+
firstName|length(firstName)|count(1)|
+---------+-----------------+--------+
 Yolonda| 7| 1283|
 Joyce| 5| 1232|
 Kecia| 5| 1300|
+---------+-----------------+--------+
only showing top 3 rows

In [0]:
peopleDF.groupBy("firstName", f.length('firstName')).agg(f.count("*").alias('count')).show(3)

+---------+-----------------+-----+
firstName|length(firstName)|count|
+---------+-----------------+-----+
 Yolonda| 7| 1283|
 Joyce| 5| 1232|
 Kecia| 5| 1300|
+---------+-----------------+-----+
only showing top 3 rows

In [0]:
peopleDF.groupBy("firstName", f.length('firstName')).count().show(3)

+---------+-----------------+-----+
firstName|length(firstName)|count|
+---------+-----------------+-----+
 Yolonda| 7| 1283|
 Joyce| 5| 1232|
 Kecia| 5| 1300|
+---------+-----------------+-----+
only showing top 3 rows

In [0]:
peopleDF.groupBy('salary').agg(f.count('*')).show(3)

+------+--------+
salary|count(1)|
+------+--------+
 92080| 141|
 29814| 15|
 43527| 68|
+------+--------+
only showing top 3 rows

In [0]:
peopleDF.agg(f.min('salary'), f.max('salary'), f.avg('salary'), f.sum('salary')).show()

+-----------+-----------+-------------+------------+
min(salary)|max(salary)| avg(salary)| sum(salary)|
+-----------+-----------+-------------+------------+
 -26884| 180841|72633.0076033|726330076033|
+-----------+-----------+-------------+------------+

In [0]:
peopleDF.groupBy('firstName').agg(f.min('salary')).orderBy('firstName').show(3)

+---------+-----------+
firstName|min(salary)|
+---------+-----------+
 Aaron| 3829|
 Abbey| 10320|
 Abbie| 3212|
+---------+-----------+
only showing top 3 rows

In [0]:
peopleDF.groupBy('firstName', 'gender').agg(f.count('*'), f.min('salary')).orderBy('firstName').show(3)

+---------+------+--------+-----------+
firstName|gender|count(1)|min(salary)|
+---------+------+--------+-----------+
 Aaron| M| 4020| 3829|
 Abbey| F| 1273| 10320|
 Abbie| F| 1288| 3212|
+---------+------+--------+-----------+
only showing top 3 rows

In [0]:
AaronSalary = peopleDF.filter("firstName == 'Aaron'").groupBy('salary').count().orderBy('salary')
AaronSalary.show(3)

+------+-----+
salary|count|
+------+-----+
 3829| 1|
 7488| 1|
 7997| 1|
+------+-----+
only showing top 3 rows

In [0]:
peopleDF.orderBy('firstName').groupBy('firstName')\
        .agg(f.count("*").alias('count'))\
        .orderBy(f.col('count').desc())\
        .show(3)

+---------+-----+
firstName|count|
+---------+-----+
 Sammie| 5462|
 Daryl| 5442|
 Bobbie| 5432|
+---------+-----+
only showing top 3 rows

In [0]:
peopleDF.orderBy('firstName').groupBy('firstName')\
        .agg(f.count("*").alias('count'))\
        .orderBy(f.col('count'))\
        .show(3)

+---------+-----+
firstName|count|
+---------+-----+
 Marylynn| 1150|
 Verna| 1154|
 Carlena| 1159|
+---------+-----+
only showing top 3 rows